In [ ]:
import pandas as pd
import os
import numpy as np

import seaborn as sns 
from matplotlib import pyplot as plt

%matplotlib inline
%config InlineBackend.figure_format = 'retina' # optionally, you can change 'svg' to 'retina'



In [ ]:
# utils 

joinpath = os.path.join


def create_dir(path): 
    if not os.path.exists(path): 
        os.mkdir(path)

In [ ]:
# definitions 

FITNESS_MEAN = 'fitness_mean'
FITNESS_SD = 'fitness_sd'
POP_SIZE = 'pop_size'
GENS = 'gens'
REPRESENTATION = 'representation'

ANALYSIS_DIR = 'analysis/'
create_dir(ANALYSIS_DIR)




In [ ]:
overview_mr = pd.read_csv('results_mr/overview_mr.csv', sep=';')
overview_mr.insert(0, 'run_name', [f'{run}_mr' for run in overview_mr.run_id])
#[f'{run}_mr' for run in overview_mr.run_id]
overview_mr

In [ ]:
overview_dp = pd.read_csv(os.path.join('results','overview.csv'), sep=';')
overview_dp.insert(0, 'run_name', [f'{run}_dp' for run in overview_dp.run_id])

overview_dp

In [ ]:
overview = pd.concat([overview_dp, overview_mr], axis=0)
overview.info()

In [ ]:
overview.info()

In [ ]:
overview['fitness_mean']

In [ ]:
overview.loc[overview.run_id == 23, ].sort_values(by='fitness_mean').head(10)

In [ ]:


configdims = [
    'co_p', 'crossover', 'difficulty', 'diversity_measure','early_stopping_patience',
    'elitism', 'epochs', 'fitness_sharing','gens', 'mu_p', 
    'mutation', 'optim', 'pop_size', 'representation','selection'
]

techdims = [
    'user_id', 'comments'
]

iddims = [
    'run_name', 'run_id', 'gs_id'
]

metricdims = [
    'duration', 'fitness_mean', 'fitness_sd',
       'stopped_early'
]


def analyse_config(overview, configdims, analysis_name): 
    outpath = joinpath(ANALYSIS_DIR, analysis_name)
    create_dir(outpath)

    configdims_count = overview[configdims].apply(pd.Series.nunique)
    configdims_vary = configdims_count[configdims_count != 1].index.tolist()
    configdims_constant = configdims_count[configdims_count == 1].index.tolist()

    configdims_vary

    configdims_vary = overview[configdims_vary].apply(pd.Series.unique).reset_index().rename(columns = {'index': 'param', 0: 'values'})
    configdims_vary
    configdims_vary.to_csv(joinpath(outpath, 'configdims_vary.csv'), index=False, sep=';')


    configdims_constant = overview[configdims_constant].apply(pd.Series.unique).reset_index().rename(columns = {'index': 'param', 0: 'values'})
    configdims_constant
    configdims_constant.to_csv(joinpath(outpath, 'configdims_constant.csv'), index=False, sep=';')

    info = pd.DataFrame({
        'n_combos': [overview.shape[0]]
    })

    
    info.to_csv(joinpath(outpath, 'info.csv'), index=False, sep=';')

    
    print(f'N_combos: {overview.shape[0]}')
    print(f'configdims_constant:\n{configdims_constant}')
    print(f'configdims_vary:\n{configdims_vary}')

    
    return configdims_vary.param.tolist()
    

# grid run 0 
- compare selection 
- pop_size

In [ ]:
ex0 = overview.loc[overview.run_name == '0_dp']

ex0.sort_values(by=[REPRESENTATION, FITNESS_MEAN])

In [ ]:
analyse_config(ex0, configdims, 'ex0_popSize_selectio')

In [ ]:
repr_grid1 = pd.pivot(ex1, index=configdims_vary_g1[0:-1], columns=['representation'], values=[fitness_mean])
repr_grid1_comp = repr_grid1.idxmin(axis=1).values.tolist()
repr_grid1_comp

repr_grid1.idxmin(axis=1).value_counts()

# first grid run


In [ ]:
grid1 = overview.loc[(overview.run_name.isin(['24_dp'])),:]

configdims_vary_g1 = analyse_config(
    overview=grid1,
    configdims=configdims,
    analysis_name='01_grid1')




## representation

In [ ]:
# representation 

fig, ax = plt.subplots(1,1)
#group = (plotdata.representation == 'maintain_init_puzzle').values
#group = [1 if row.representation == 'maintain_init_puzzle' else 0 for row in plotdata.iterrows()]
group = 'representation'
h_order = grid1.groupby(group)[FITNESS_MEAN].mean().sort_values().index.tolist()
h_order
sns.boxplot(data = grid1, y=fitness_mean, x=group, hue=group,hue_order=h_order, ax=ax)
#ax.set_xlabel('', rotation=45)
ax.tick_params(labelrotation=45)



In [ ]:
configdims_vary_g1[0:-1]
grid1.groupby(configdims_vary_g1[0:-1]).representation

In [ ]:
pd.melt(grid1, id_vars=configdims_vary_g1[0:-1], value_vars=[fitness_mean, 'representation'])

In [ ]:
repr_grid1 = pd.pivot(grid1.loc[grid1.run_name=='24_dp', :], index=configdims_vary_g1[0:-1], columns=['representation'], values=[fitness_mean])
repr_grid1_comp = repr_grid1.idxmin(axis=1).values.tolist()
repr_grid1_comp

repr_grid1.idxmin(axis=1).value_counts()

In [ ]:
def cetris_paribus(overview_df, configdims_vary, dim, verbose=False): 
    cols_hold_constant = [d for d in configdims_vary if d not in [dim]] 
    
    repr_grid1 = pd.pivot(overview_df, index=cols_hold_constant, columns=[dim], values=[FITNESS_MEAN])
    if verbose: print(repr_grid1)
    repr_grid1_comp = repr_grid1.idxmin(axis=1).values.tolist()
    repr_grid1_comp

    return repr_grid1.idxmin(axis=1).value_counts()

In [ ]:
cetris_paribus(overview_df=grid1, configdims_vary=configdims_vary_g1, dim=REPRESENTATION, verbose=True)


In [ ]:
overview_df=grid1
configdims_vary=configdims_vary_g1
dim=REPRESENTATION

#cetris_paribus(overview_df=grid1, configdims_vary=configdims_vary_g1, dim=REPRESENTATION)

cols_hold_constant = [d for d in configdims_vary if d not in [dim]] 
cols_hold_constant
configdims_vary
pd.pivot(overview_df, index=cols_hold_constant, columns=[dim], values=[FITNESS_MEAN])


## maintain init vs random 
- best of each
- compare hostory

In [ ]:
# TODO:

## ex2: explore maintain_init_position
- subset of ex1 PLUS Sudoku specific operators (['24_dp', '12_mr'])

In [ ]:
ex2 = overview.loc[
    (overview.run_name.isin(['24_dp', '12_mr']))
    & (overview.representation == 'maintain_init_puzzle')
    ,:]

configdims_vary_ex2 = analyse_config(ex2, configdims, 'ex2-maintain')

### overall variance


In [ ]:

fig, (ax1, ax2) = plt.subplots(ncols=2)
sns.boxplot(data=ex2, y=FITNESS_MEAN, ax=ax1)
sns.boxplot(data=ex2, y=FITNESS_SD, ax=ax2)
fig.tight_layout()

In [ ]:
ex2.sort_values(by=FITNESS_MEAN, axis=0, ascending=True).head(20)

In [ ]:
for dim in configdims_vary_ex2: 
    print(dim)
    res = cetris_paribus(ex2, configdims_vary_ex2, dim)
    print(res)


In [ ]:
    # plot scores 
    def plot_scores(scores, explorations_path, filename, width, height, dodge): 
        my_dpi = 200
        fig = plt.figure(
            figsize=(
                #10, 8
                width/my_dpi, height/my_dpi
            )
        )
        ax = sns.pointplot(data=scores, x='model', y='mean', hue='config', alpha=.7, dodge=dodge, join=False, scale=.5)

        # Find the x,y coordinates for each point
        x_coords = []
        y_coords = []
        for point_pair in ax.collections:
            for x, y in point_pair.get_offsets():
                x_coords.append(x)
                y_coords.append(y)

        # Calculate the type of error to plot as the error bars
        # Make sure the order is the same as the points were looped over
        #errors = tips.groupby(['smoker', 'sex']).std()['tip']
        #colors = ['steelblue']*2 + ['coral']*2
        ax.errorbar(x_coords, y_coords, yerr=scores.sd, fmt=' ', zorder=-1, color='black', capsize=2)

        plt.legend(bbox_to_anchor=(0., 1.02, 1., .102), loc='lower left',
                   ncol=1, mode="expand", borderaxespad=0., prop={'size': 6})
        plt.tight_layout()
        plt.xticks(rotation=90)
        ax.set(ylabel='mean micro f1 score')


        plt.savefig(os.path.join(explorations_path, f'{filename}.png'), dpi=200, bbox_inches = "tight")


        plt.show()
       
    # generate and save plot
    plot_scores(scores, explorations_path, 'comp_all', 1200, 1400, .4)

# pop size

In [ ]:
from copy import deepcopy

In [ ]:
def filter_runs(overview, colname, values):
    df = deepcopy(overview)
    if not isinstance(values, list): 
        values = [values]
    return overview.loc[overview[colname].isin(values), ]
    

In [ ]:
plotdata = filter_runs(overview, 'run_id', 0)
plotdata.info()

In [ ]:
plotdata.loc['cat_maintain_init_puzzle'] = np.where(plotdata.representation == 'maintain_init_puzzle', 1, 0)

In [ ]:
plotdata

In [ ]:

fig, ax = plt.subplots(1,1)
#group = (plotdata.representation == 'maintain_init_puzzle').values
group = [1 if row.representation == 'maintain_init_puzzle' else 0 for row in plotdata.iterrows()]
group
#sns.boxplot(data = plotdata, y=fitness_mean, x=group, hue=group, ax=ax)


In [ ]:
for row in plotdata.iteritems(): 
    print(row)

In [ ]:
plotdata[(plotdata.fitness_mean < 40) & (plotdata.pop_size == 100)]

In [ ]:
overview.loc[overview['run_id'] in [0] , ]